In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

In [ ]:
mitsar_chls

In [2]:
import matplotlib.pyplot as plt
from torch.utils.data import Dataset

import numpy as np
import pandas as pd
import os

In [3]:
from scipy import signal

from scipy.signal import resample
from scipy.signal import butter, lfilter
from scipy.signal import butter, sosfilt, sosfreqz
from scipy.signal import freqz

In [4]:
import mne

In [5]:
annotation_meaning = {
    'S  4': 'baseline_start',
    'S 44': 'baseline_end',
    'S  5': 'rest_start', 
    'S 50': 'rest_end',
    'S  6': 'scroll_start',
    'S 60': 'scroll_end',
    'S  7': 'math_start',
    'S 70': 'math_end',
    'S 66': 'scroll_sample_start', 
    'S 69': 'scroll_sample_end',
    'S 77': 'math_sample_start',
    'S 79': 'math_sample_end'
}

In [6]:
channels_groups = {
    'Fp1': ['Fp1','FP2'],
    'FP2': ['Fp1','FP2'],
    'F7': ['F7','F3','FC5'],
    'F3': ['F7','F3','FC5'],
    'FC5': ['F7','F3','FC5'], 
    'Fz': ['Fz','FC1','FC2'],
    'FC1': ['Fz','FC1','FC2'],
    'FC2': ['Fz','FC1','FC2'], 
    'F4': ['F4','F8','FC6'],
    'F8': ['F4','F8','FC6'],
    'FC6': ['F4','F8','FC6'],
    'T7': ['T7','CP5','P7'],
    'CP5': ['T7','CP5','P7'],
    'P7': ['T7','CP5','P7'], 
    'T8': ['T8','CP6','P8'],
    'CP6': ['T8','CP6','P8'],
    'P8': ['T8','CP6','P8'], 
    'C3': ['C3','Cz','C4'],
    'Cz': ['C3','Cz','C4'],
    'C4': ['C3','Cz','C4'], 
    'P3': ['P3','Pz','P4','CP1','CP2'],
    'Pz': ['P3','Pz','P4','CP1','CP2'],
    'P4': ['P3','Pz','P4','CP1','CP2'],
    'CP1': ['P3','Pz','P4','CP1','CP2'],
    'CP2': ['P3','Pz','P4','CP1','CP2'], 
    'O1': ['O1','Oz','O2'],
    'Oz': ['O1','Oz','O2'],
    'O2': ['O1','Oz','O2']
}

In [7]:
channels = ['Fp1', 'FP2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'C3', 'Cz', 'C4', \
            'P3', 'Pz', 'P4', 'O1', 'O2']


In [8]:
channels_to_interpolate = {
    23: [],
    24: ['O2', 'F7', 'Fz'],
    25: ['Fc2', 'T7', 'C3'],
    26: ['T7', 'Fc2', 'C3'],
    27: ['C3'],
    28: ['Fc2', 'C3', 'T7'],
    29: [],
    30: ['O2', 'T7', 'Fc2', 'C3'],
    31: ['T7', 'Fc2', 'C3'],
    32: ['O2'],
    33: ['O2', 'Oz', 'P8', 'Fc5', 'T7', 'Fc2'],
    34: ['O2'],
    35: ['Fc2', 'C3', 'T7'],
    36: ['T7', 'C3', 'Fc2'],
    37: ['T7'],
    38: ['Fz', 'T7', 'C3', 'Fc2'],
    39: ['O2'],
    40: [],
    41: ['T7', 'C3', 'Fc2'],
    42: ['O2', 'Fz'],
    43: ['T7', 'C3', 'Fc2'],
    44: ['F8', 'Fc2', 'T7', 'C4'],
    45: ['Oz', 'P4', 'P7'],
    46: ['Fc2', 'Fc1', 'O2', 'T7'],
    47: ['O2', 'F7', 'Fz'],
    48: ['F7', 'Fz', 'Fc1'],
    49: ['O2', 'F7', 'Fz'],
    50: ['F7', 'Fz'],
    51: ['T8'],
    52: ['Fp1', 'Fc2']
}

In [9]:
channels_to_interpolate = {k: [j for j in v if j in channels] for k, v in channels_to_interpolate.items()}

In [10]:
channels_groups = {k: [j for j in v if j in channels] for k, v in channels_groups.items() if k in channels}

In [11]:
# channels = ['Fp1','FP2', 'F7','F3','FC5', 'Fz','FC1','FC2', 'F4','F8','FC6', \
#             'T7','CP5','P7', 'T8','CP6','P8', 'C3','Cz','C4', 'P3','Pz','P4','CP1','CP2', 'O1','Oz','O2']


In [ ]:
!ls /home/data/HSE_math_exp_2/raw/

100_math-edf.edf  39_math-edf.edf  60_math-edf.edf  81_math-edf.edf
101_math-edf.edf  40_math-edf.edf  61_math-edf.edf  82_math-edf.edf
102_math-edf.edf  41_math-edf.edf  62_math-edf.edf  83_math-edf.edf
103_math-edf.edf  42_math-edf.edf  63_math-edf.edf  84_math-edf.edf
104_math-edf.edf  43_math-edf.edf  64_math-edf.edf  85_math-edf.edf
23_math-edf.edf   44_math-edf.edf  65_math-edf.edf  86_math-edf.edf
24_math-edf.edf   45_math-edf.edf  66_math-edf.edf  87_math-edf.edf
25_math-edf.edf   46_math-edf.edf  67_math-edf.edf  88_math-edf.edf
26_math-edf.edf   47_math-edf.edf  68_math-edf.edf  89_math-edf.edf
27_math-edf.edf   48_math-edf.edf  69_math-edf.edf  90_math-edf.edf
28_math-edf.edf   49_math-edf.edf  70_math-edf.edf  91_math-edf.edf
29_math-edf.edf   50_math-edf.edf  71_math-edf.edf  92_math-edf.edf
30_math-edf.edf   51_math-edf.edf  72_math-edf.edf  93_math-edf.edf
31_math-edf.edf   52_math-edf.edf  73_math-edf.edf  94_math-edf.edf
32_math-edf.edf   53_math-edf.edf  74_math-edf.e

In [14]:
users = range(23, 105)

In [ ]:
annotations

(array([[ 20440,      0,      1],
        [ 20701,      0,      5],
        [ 22375,      0,      2],
        [ 52290,      0,      6],
        [ 52340,      0,      3],
        [ 52390,      0,      8],
        [ 53381,      0,      9],
        [ 54798,      0,      8],
        [ 55396,      0,      9],
        [ 56554,      0,      8],
        [ 57460,      0,      9],
        [ 58910,      0,      8],
        [ 59700,      0,      9],
        [ 60804,      0,      8],
        [ 61568,      0,      9],
        [ 62418,      0,      8],
        [ 63508,      0,      9],
        [ 64378,      0,      8],
        [ 64634,      0,      9],
        [ 65479,      0,      8],
        [ 66048,      0,      9],
        [ 67027,      0,      8],
        [ 67929,      0,      9],
        [ 68628,      0,      8],
        [ 69506,      0,      9],
        [ 71035,      0,      8],
        [ 71566,      0,      9],
        [ 72653,      0,      8],
        [ 73130,      0,      9],
        [ 7416

In [15]:
signal_data = []

for user in users:
    try:
        data_test = mne.io.read_raw_edf('/home/data/HSE_math_exp_2/raw/{}_math-edf.edf'.format(user), preload=True)
        data_test = data_test.filter(0.9, 40)
        annotations = mne.events_from_annotations(data_test)
        remap = {v: k for k, v in annotations[1].items()}
        annotations_remaped = []
        for i in annotations[0]:
            annotations_remaped.append((i[0], i[1], remap.get(i[2])))

        annotations_remaped = [i for i in annotations_remaped if i[2] in ['S  7', 'S 70', 'S  6', 'S 60', 'S  5', 'S 50']]
        data_eeg = data_test.to_data_frame()[channels]
        ch_names = data_eeg.columns
        if user in channels_to_interpolate:
            for ch_name in ch_names:
                if ch_name in channels_to_interpolate[user]:
                    print(ch_name)
                    ch_for_interpol = [i for i in channels_groups[ch_name] if i not in channels_to_interpolate[user]]

                    if len(ch_for_interpol) == 0:
                        print('W')
                        ch_for_interpol = [i for i in channels_groups if i not in channels_to_interpolate[user]]

                    chs_vals = data_eeg[ch_for_interpol].values.mean(-1)

                    data_eeg[ch_name] = chs_vals

        data_sliced = []
        for i, _ in enumerate(annotations_remaped):
            if annotations_remaped[i][2] == 'S 50':
                start = annotations_remaped[i-1][0]
                end = annotations_remaped[i][0]
                label = 0
                dur = (end - start) / 1000
                data_sliced.append((start, end, label, dur))

            if annotations_remaped[i][2] == 'S 60':
                start = annotations_remaped[i-1][0]
                end = annotations_remaped[i][0]
                label = 1
                dur = (end - start) / 1000
                data_sliced.append((start, end, label, dur))

            if annotations_remaped[i][2] == 'S 70':
                start = annotations_remaped[i-1][0]
                end = annotations_remaped[i][0]
                label = 2
                dur = (end - start) / 1000
                data_sliced.append((start, end, label, dur))

            # if annotations_remaped[i][2] == 'S 44':
            #     start = annotations_remaped[i-1][0]
            #     end = annotations_remaped[i][0]
            #     label = -1
            #     dur = (end - start) / 1000
            #     data_sliced.append((start, end, label, dur))

        data_sliced_window = []
        for s in data_sliced:
            if s[2] == 0:
                counter = 0
                for i in range(1500, int(s[3] * 1000), 1500):
                    data_sliced_window.append((s[0] + i - 1500, s[0] + i, s[2], 1.5))
                    counter += 1
                    # if counter > 50:
                    #     break

            else:
                counter = 0
                for i in range(1500, int(s[3] * 1000), 1500):
                    data_sliced_window.append((s[0] + i - 1500, s[0] + i, s[2], 1.5))
                    counter += 1
                    # for i in range(3000, int(s[3] * 1000), 1000):
    #                 i = 1500 # fix to only one fragment from each sample
    #                 data_sliced_window.append((s[0] + i - 1500, s[0] + i, s[2], 1.5))


        for j, (start, end, label, dur) in enumerate(data_sliced_window):
            signal_data.append({
                'data': data_eeg.iloc[start:end].values,
                'label': label,
                'data_names': data_eeg.columns,
                'user': user,
                'seq_id': j
            })
    except:
        raise

Extracting EDF parameters from /home/data/HSE_math_exp_2/raw/23_math-edf.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 2620499  =      0.000 ...  5240.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.9 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.90
- Lower transition bandwidth: 0.90 Hz (-6 dB cutoff frequency: 0.45 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1835 samples (3.670 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.3s remaining:    0.0s


Used Annotations descriptions: ['New Segment', 'S  4', 'S  5', 'S  6', 'S  7', 'S  8', 'S 44', 'S 50', 'S 60', 'S 63', 'S 66', 'S 69', 'S 70', 'S 73', 'S 77', 'S 79']


[Parallel(n_jobs=1)]: Done  37 out of  37 | elapsed:    2.3s finished


Extracting EDF parameters from /home/data/HSE_math_exp_2/raw/24_math-edf.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 2307999  =      0.000 ...  4615.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.9 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.90
- Lower transition bandwidth: 0.90 Hz (-6 dB cutoff frequency: 0.45 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1835 samples (3.670 s)



...

[Parallel(n_jobs=1)]: Done  37 out of  37 | elapsed:    1.9s finished


Extracting EDF parameters from /home/data/HSE_math_exp_2/raw/104_math-edf.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 2203499  =      0.000 ...  4406.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.9 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.90
- Lower transition bandwidth: 0.90 Hz (-6 dB cutoff frequency: 0.45 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1835 samples (3.670 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s


Used Annotations descriptions: ['New Segment', 'S  4', 'S  5', 'S  6', 'S  7', 'S  8', 'S 44', 'S 50', 'S 60', 'S 63', 'S 66', 'S 69', 'S 70', 'S 73', 'S 77', 'S 79']


[Parallel(n_jobs=1)]: Done  37 out of  37 | elapsed:    2.0s finished


In [16]:
user

104

In [17]:
len(signal_data)

89563

In [18]:
len([i for i in signal_data if i['user'] == 23])

1126

In [19]:
([i for i in signal_data if i['user'] == 26])[-5:]

[{'data': array([[ 45.46062804,  41.29250618,  26.08463208, ...,  23.0980014 ,
           19.51918328,  23.65399856],
         [ 54.34388565,  48.39412519,  29.92511675, ...,  24.98553584,
           19.46156251,  24.99460616],
         [ 64.04373965,  56.57542362,  34.72641646, ...,  27.18678447,
           20.39335659,  26.8197511 ],
         ...,
         [-62.87811417, -61.888331  , -23.95848737, ...,  -6.39115395,
           -9.43721376,  -4.81246541],
         [-62.82275947, -62.75187442, -25.3389503 , ...,  -8.37036553,
           -9.658329  ,  -5.30323422],
         [-62.63288673, -63.65175425, -26.28869278, ..., -10.14261874,
           -9.63153987,  -5.61342532]]),
  'label': 0,
  'data_names': Index(['Fp1', 'FP2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'C3', 'Cz', 'C4', 'P3',
         'Pz', 'P4', 'O1', 'O2'],
        dtype='object'),
  'user': 26,
  'seq_id': 1084},
 {'data': array([[-62.46630002, -64.39399223, -26.58835358, ..., -11.38867243,
           -9.26315632,  -5.61012243],
 

In [20]:
# np.save('/home/data/HSE_math_exp_2/processing_internal/processed_data.3s.npy', signal_data)

In [21]:
ppp = []

for sample in signal_data:
    user_id = sample['user']
    # path = '/home/data/HSE_math_exp_2/processing_internal.v2/sliced.limited_ch'.format(user_id)
    path_file = '/home/data/HSE_math_exp_2/processing_internal.v2/sliced.limited_ch/x_{}_{}.npy'.format(user_id, sample['seq_id'])

    # os.makedirs(path, exist_ok=True)
    
    np.save(path_file, sample['data'])

In [22]:
ys = {}

for sample in signal_data:
    if ys.get(sample['user']) is None:
        ys[sample['user']] = []
        
    ys[sample['user']].append(sample['label'])

In [23]:
for user_id in ys:
    path = '/home/data/HSE_math_exp_2/processing_internal.v2/sliced.limited_ch/{}_y.npy'.format(user_id)
    np.save(path, ys[user_id])

In [56]:
ttt = os.listdir('/home/data/HSE_math_exp_2/processing_internal.v2/sliced.limited_ch')

In [57]:
len(ttt)

89646

In [16]:
!ls /home/data/HSE_VALIDATION/1/raw

1  10  11  2  3  4  5  6  7  8	9  raw.7z


In [20]:
!ls /home/data/HSE_VALIDATION/1/raw/11/maths/

 116
 230703_0155_EEG.csv
 debug_E046_20230703175400_null.zip
 debug_R_E046_20230703175400_null.csv
'_numbers_pilot_3.2 (1-2-3)_2023-07-03_17h54.46.864.csv'
'_numbers_pilot_3.2 (1-2-3)_2023-07-03_17h54.46.864.psydat'
 Subject_events.csv
 Subject_output.xlsx


In [21]:
paths = [
    '/home/data/HSE_VALIDATION/1/raw/1/maths/230609_0144_EEG.csv',
    '/home/data/HSE_VALIDATION/1/raw/2/maths/230614_0143_EEG.csv',
    '/home/data/HSE_VALIDATION/1/raw/3/maths/230615_0145_EEG.csv',
    '/home/data/HSE_VALIDATION/1/raw/4/maths/230615_0146_EEG.csv',
    '/home/data/HSE_VALIDATION/1/raw/5/maths/230616_0146_EEG.csv',
    '/home/data/HSE_VALIDATION/1/raw/6/maths/230620_0147_EEG.csv',
    '/home/data/HSE_VALIDATION/1/raw/7/maths/230623_0149_EEG.csv',
    '/home/data/HSE_VALIDATION/1/raw/8/maths/230630_0150_EEG.csv',
    '/home/data/HSE_VALIDATION/1/raw/9/maths/230701_0152_EEG.csv',
    '/home/data/HSE_VALIDATION/1/raw/10/maths/230703_0153_EEG.csv',
    '/home/data/HSE_VALIDATION/1/raw/11/maths/230703_0155_EEG.csv',
]

In [22]:
signal_data_inner = []

In [23]:
for count, path in enumerate(paths):
    count = count+1
    ch_names = ['Fp1', 'FP2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'VEOG', 'C3', 'Cz', 'C4', \
                'HEOG', 'T5', 'P3', 'Pz', 'P4', 'T6', 'O1', 'O2', 'A1', 'A2', \
                'ECG_l', 'ECG_r']

    ch_types = ['eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eog', 'eeg', 'eeg', 'eeg', \
               'eog', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'ecg', 'ecg', ]

    evt = pd.read_csv('/home/data/HSE_VALIDATION/1/processed/{}/events_maths_{}.csv'.format(count, count))
    annot = mne.annotations_from_events(evt.values[:, :3], sfreq=256)
    annot.to_data_frame()['description'][0]

    event_id = {
        '4':'S  4',
        '44':'S 40',
        '1':'S  5',
        '10':'S 50',
        '3':'S  7',
        '30':'S 70',
        '2':'S  6',
        '20':'S 60',
        '22':'S 63',
        '33':'S 73',
        '29':'S 69',
        '39':'S 79'
    }

    annot.rename(event_id)


    # Load data
    _data_raw = pd.read_csv(path, sep=';', index_col=False)
    _data_raw = _data_raw.drop(columns=['REF'])
    sfreq = 256 
    info = mne.create_info(ch_names = ch_names, sfreq = sfreq)
    data_raw = mne.io.RawArray(_data_raw.values[:, :-4].T, info)
    #data_raw = mne.io.read_raw(os.path.join(init_dir_path, edf_fname), preload=True)
    samp_rate = data_raw.info['sfreq']
    data_raw.set_annotations(annot)
    data_raw.set_channel_types(dict(zip(ch_names, ch_types)))
    data_test = data_raw

    user = count + 9

    data_test = data_test.filter(0.9, 40)
    annotations = mne.events_from_annotations(data_test)
    remap = {v: k for k, v in annotations[1].items()}
    annotations_remaped = []
    for i in annotations[0]:
        annotations_remaped.append((i[0], i[1], remap.get(i[2])))

    annotations_remaped = [i for i in annotations_remaped if i[2] in ['S  7', 'S 70', 'S  6', 'S 60', 'S  5', 'S 50']]
    data_eeg = data_test.to_data_frame()[channels]
    ch_names = data_eeg.columns
    # for ch_name in ch_names:
    #     if ch_name in channels_to_interpolate[user]:
    #         print(ch_name)
    #         ch_for_interpol = [i for i in channels_groups[ch_name] if i not in channels_to_interpolate[user]]

    #         if len(ch_for_interpol) == 0:
    #             print('W')
    #             ch_for_interpol = [i for i in channels_groups if i not in channels_to_interpolate[user]]

    #         chs_vals = data_eeg[ch_for_interpol].values.mean(-1)

    #         data_eeg[ch_name] = chs_vals

    data_sliced = []
    for i, _ in enumerate(annotations_remaped):
        if annotations_remaped[i][2] == 'S 50':
            start = annotations_remaped[i-1][0]
            end = annotations_remaped[i][0]
            label = 0
            dur = (end - start) / 1000
            data_sliced.append((start, end, label, dur))

        if annotations_remaped[i][2] == 'S 60':
            start = annotations_remaped[i-1][0]
            end = annotations_remaped[i][0]
            label = 1
            dur = (end - start) / 1000
            data_sliced.append((start, end, label, dur))

        if annotations_remaped[i][2] == 'S 70':
            start = annotations_remaped[i-1][0]
            end = annotations_remaped[i][0]
            label = 2
            dur = (end - start) / 1000
            data_sliced.append((start, end, label, dur))

        # if annotations_remaped[i][2] == 'S 44':
        #     start = annotations_remaped[i-1][0]
        #     end = annotations_remaped[i][0]
        #     label = -1
        #     dur = (end - start) / 1000
        #     data_sliced.append((start, end, label, dur))

    data_sliced_window = []
    for s in data_sliced:
        if s[2] == 0:
            counter = 0
            for i in range(1500, int(s[3] * 1000), 1500):
                data_sliced_window.append((s[0] + i - 1500, s[0] + i, s[2], 1.5))
                counter += 1
                # if counter > 50:
                #     break

        else:
            counter = 0
            for i in range(1500, int(s[3] * 1000), 1500):
                data_sliced_window.append((s[0] + i - 1500, s[0] + i, s[2], 1.5))
                counter += 1
                # for i in range(3000, int(s[3] * 1000), 1000):
    #                 i = 1500 # fix to only one fragment from each sample
    #                 data_sliced_window.append((s[0] + i - 1500, s[0] + i, s[2], 1.5))


    for j, (start, end, label, dur) in enumerate(data_sliced_window):
        signal_data_inner.append({
            'data': data_eeg.iloc[start:end].values,
            'label': label,
            'data_names': data_eeg.columns,
            'user': user,
            'seq_id': j
        })

/tmp/ipykernel_3742159/1502586669.py:33: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  _data_raw = pd.read_csv(path, sep=';', index_col=False)


Creating RawArray with float64 data, n_channels=23, n_times=378331
    Range : 0 ... 378330 =      0.000 ...  1477.852 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.9 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.90
- Lower transition bandwidth: 0.90 Hz (-6 dB cutoff frequency: 0.45 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 939 samples (3.668 s)



/tmp/ipykernel_3742159/1502586669.py:41: RuntimeWarning: The unit for channel(s) A1, A2, C3, C4, Cz, ECG_l, ECG_r, F3, F4, F7, F8, FP2, Fp1, Fz, HEOG, O1, O2, P3, P4, Pz, T5, T6, VEOG has changed from NA to V.
  data_raw.set_channel_types(dict(zip(ch_names, ch_types)))
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Used Annotations descriptions: ['S  4', 'S  5', 'S  6', 'S  7', 'S 40', 'S 50', 'S 60', 'S 63', 'S 69', 'S 70', 'S 73', 'S 79']


[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.2s finished
/tmp/ipykernel_3742159/1502586669.py:33: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  _data_raw = pd.read_csv(path, sep=';', index_col=False)


...

Creating RawArray with float64 data, n_channels=23, n_times=382414
    Range : 0 ... 382413 =      0.000 ...  1493.801 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.9 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.90
- Lower transition bandwidth: 0.90 Hz (-6 dB cutoff frequency: 0.45 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 939 samples (3.668 s)

Used Annotations descriptions: ['S  4', 'S  5', 'S  6', 'S  7', 'S 40', 'S 50', 'S 60', 'S 63', 'S 69', 'S 70', 'S 73', 'S 79']


/tmp/ipykernel_3742159/1502586669.py:41: RuntimeWarning: The unit for channel(s) A1, A2, C3, C4, Cz, ECG_l, ECG_r, F3, F4, F7, F8, FP2, Fp1, Fz, HEOG, O1, O2, P3, P4, Pz, T5, T6, VEOG has changed from NA to V.
  data_raw.set_channel_types(dict(zip(ch_names, ch_types)))
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.2s finished
/tmp/ipykernel_3742159/1502586669.py:33: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  _data_raw = pd.read_csv(path, sep=';', index_col=False)


Creating RawArray with float64 data, n_channels=23, n_times=402210
    Range : 0 ... 402209 =      0.000 ...  1571.129 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.9 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.90
- Lower transition bandwidth: 0.90 Hz (-6 dB cutoff frequency: 0.45 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 939 samples (3.668 s)



/tmp/ipykernel_3742159/1502586669.py:41: RuntimeWarning: The unit for channel(s) A1, A2, C3, C4, Cz, ECG_l, ECG_r, F3, F4, F7, F8, FP2, Fp1, Fz, HEOG, O1, O2, P3, P4, Pz, T5, T6, VEOG has changed from NA to V.
  data_raw.set_channel_types(dict(zip(ch_names, ch_types)))
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.2s finished


Used Annotations descriptions: ['S  4', 'S  5', 'S  6', 'S  7', 'S 40', 'S 50', 'S 60', 'S 63', 'S 69', 'S 70', 'S 73', 'S 79']


In [24]:
signal_data_inner[243]

{'data': array([[ 1.12646887e+07,  2.02655342e+07, -4.29387460e+07, ...,
          2.98799132e+06,  3.67979723e+05, -7.68434059e+05],
        [ 8.22057176e+06,  1.79300361e+07, -4.52540289e+07, ...,
          1.47723531e+06,  2.19553885e+06,  1.33342516e+06],
        [ 6.06715095e+06,  1.92788749e+07, -4.84247353e+07, ...,
          8.59716611e+05,  4.13128396e+06,  2.82721812e+06],
        ...,
        [-9.45479507e+07, -9.96031074e+07, -5.08141147e+07, ...,
         -1.69322541e+07, -1.04194646e+07, -9.80647846e+06],
        [-9.87040152e+07, -9.84688990e+07, -4.76863256e+07, ...,
         -1.33277044e+07, -6.76279282e+06, -5.97610820e+06],
        [-1.01444081e+08, -9.71823775e+07, -4.45014545e+07, ...,
         -1.00536404e+07, -2.44043061e+06, -1.27188261e+06]]),
 'label': 2,
 'data_names': Index(['Fp1', 'FP2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'C3', 'Cz', 'C4', 'P3',
        'Pz', 'P4', 'O1', 'O2'],
       dtype='object'),
 'user': 10,
 'seq_id': 243}

In [26]:
for sample in signal_data_inner:
    user_id = sample['user']
    path_file = '/home/data/HSE_math_exp_2/processing_internal.v2/sliced.limited_ch/x_{}_{}.npy'.format(user_id, sample['seq_id'])
    
    np.save(path_file, sample['data'])

In [27]:
len(signal_data_inner[0]['data_names'])

15

In [28]:
ys = {}

for sample in signal_data_inner:
    if ys.get(sample['user']) is None:
        ys[sample['user']] = []
        
    ys[sample['user']].append(sample['label'])

In [29]:
for user_id in ys:
    path = '/home/data/HSE_math_exp_2/processing_internal.v2/sliced.limited_ch/{}_y.npy'.format(user_id)
    np.save(path, ys[user_id])


In [84]:
tmp = os.listdir('/home/data/HSE_math_exp_2/processing_internal.v2/sliced.limited_ch/')

In [89]:
[i for i in tmp if '_210_' in i]

['x_210_272.npy',
 'x_210_97.npy',
 'x_210_74.npy',
 'x_210_201.npy',
 'x_210_343.npy',
 'x_210_23.npy',
 'x_210_45.npy',
 'x_210_260.npy',
 'x_210_168.npy',
 'x_210_216.npy',
 'x_210_351.npy',
 'x_210_84.npy',
 'x_210_215.npy',
 'x_210_181.npy',
 'x_210_81.npy',
 'x_210_190.npy',
 'x_210_58.npy',
 'x_210_34.npy',
 'x_210_61.npy',
 'x_210_121.npy',
 'x_210_198.npy',
 'x_210_278.npy',
 'x_210_1.npy',
 'x_210_264.npy',
 'x_210_239.npy',
 'x_210_219.npy',
 'x_210_318.npy',
 'x_210_83.npy',
 'x_210_169.npy',
 'x_210_141.npy',
 'x_210_325.npy',
 'x_210_54.npy',
 'x_210_104.npy',
 'x_210_120.npy',
 'x_210_269.npy',
 'x_210_151.npy',
 'x_210_240.npy',
 'x_210_26.npy',
 'x_210_248.npy',
 'x_210_86.npy',
 'x_210_59.npy',
 'x_210_134.npy',
 'x_210_35.npy',
 'x_210_130.npy',
 'x_210_46.npy',
 'x_210_3.npy',
 'x_210_251.npy',
 'x_210_51.npy',
 'x_210_301.npy',
 'x_210_308.npy',
 'x_210_156.npy',
 'x_210_312.npy',
 'x_210_71.npy',
 'x_210_78.npy',
 'x_210_317.npy',
 'x_210_170.npy',
 'x_210_92.npy'